In [70]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
import warnings
warnings.filterwarnings('ignore')


plt.style.use('ggplot')
sns.set_palette("viridis")
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
class DataLoader:
    """Load and preprocess climate and supply chain data"""

    def __init__(self):
        self.climate_data = None
        self.supply_chain_data = None
        self.risk_index_data = None

    def load_demo_data(self):
        """Load demo datasets for analysis"""
        try:

            self.climate_data = pd.read_csv('climate_data.csv')
            self.supply_chain_data = pd.read_csv('supply_chain_data.csv')
            self.risk_index_data = pd.read_csv('risk_index_data.csv')
            print("Demo data loaded successfully!")

        except FileNotFoundError:
            print("Demo files not found. Generating sample data...")
            self._generate_sample_data()

        return self.climate_data, self.supply_chain_data, self.risk_index_data

    def _generate_sample_data(self):
        """Generate sample data if files are not available"""
        np.random.seed(42)


        dates = pd.date_range('2015-01-01', '2022-12-31', freq='M')
        locations = ['USA', 'China', 'India', 'Brazil', 'Germany', 'Japan']

        climate_records = []
        for location in locations:
            for date in dates:
                climate_records.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'location': location,
                    'temperature': np.random.normal(20, 5),
                    'precipitation': np.random.gamma(5, 2),
                    'extreme_events': np.random.poisson(0.3)
                })

        self.climate_data = pd.DataFrame(climate_records)

        suppliers = [f"Supplier_{i:03d}" for i in range(1, 31)]
        materials = ['Petrochemicals', 'Metals', 'Plastics', 'Equipment', 'Logistics']

        supply_chain_records = []
        for supplier in suppliers:
            supply_chain_records.append({
                'supplier_id': supplier,
                'location': np.random.choice(locations),
                'material_type': np.random.choice(materials),
                'transport_mode': np.random.choice(['Sea', 'Air', 'Road', 'Rail']),
                'single_source': np.random.choice([True, False], p=[0.3, 0.7]),
                'lead_time': np.random.randint(7, 90),
                'revenue_impact': round(np.random.uniform(0.5, 8.0), 2)
            })

        self.supply_chain_data = pd.DataFrame(supply_chain_records)


        risk_records = []
        for location in locations:
            risk_records.append({
                'location': location,
                'climate_vulnerability': round(np.random.uniform(0.2, 0.9), 2),
                'infrastructure_quality': round(np.random.uniform(0.3, 0.95), 2),
                'recovery_capacity': round(np.random.uniform(0.4, 0.9), 2),
                'water_stress': round(np.random.uniform(0.1, 0.8), 2),
                'sea_level_rise': round(np.random.uniform(0.05, 0.7), 2)
            })

        self.risk_index_data = pd.DataFrame(risk_records)


        self.climate_data.to_csv('climate_data.csv', index=False)
        self.supply_chain_data.to_csv('supply_chain_data.csv', index=False)
        self.risk_index_data.to_csv('risk_index_data.csv', index=False)

        print("Sample data generated and saved!")

data_loader = DataLoader()
climate_df, supply_chain_df, risk_index_df = data_loader.load_demo_data()

print(f"\nData Shapes:")
print(f"Climate data: {climate_df.shape}")
print(f"Supply chain data: {supply_chain_df.shape}")
print(f"Risk index data: {risk_index_df.shape}")

print("\nClimate Data Sample:")
print(climate_df.head())

print("\nSupply Chain Data Sample:")
print(supply_chain_df.head())

print("\nRisk Index Data Sample:")
print(risk_index_df.head())



Demo data loaded successfully!

Data Shapes:
Climate data: (576, 5)
Supply chain data: (30, 7)
Risk index data: (6, 6)

Climate Data Sample:
         date location  temperature  precipitation  extreme_events
0  2015-01-31      USA    22.483571       8.748617               0
1  2015-02-28      USA    18.829233       8.357853               1
2  2015-03-31      USA    17.095609       7.243243               0
3  2015-04-30      USA     6.937255      14.070968               0
4  2015-05-31      USA    17.859770       6.479183               0

Supply Chain Data Sample:
    supplier_id location   material_type transport_mode  single_source  \
0  Supplier_001    China       Equipment           Rail          False   
1  Supplier_002    Japan       Logistics           Road           True   
2  Supplier_003  Germany          Metals           Rail           True   
3  Supplier_004      USA  Petrochemicals            Sea          False   
4  Supplier_005  Germany        Plastics           Road     

In [ ]:
class DataPreprocessor:
    """Preprocess and prepare data for analysis"""

    def __init__(self, climate_data, supply_chain_data, risk_index_data):
        self.climate_data = climate_data.copy()
        self.supply_chain_data = supply_chain_data.copy()
        self.risk_index_data = risk_index_data.copy()
        self.processed_data = None

    def preprocess_data(self):
        """Clean and preprocess all datasets"""

        self.climate_data['date'] = pd.to_datetime(self.climate_data['date'])

        climate_metrics = self.climate_data.groupby('location').agg({
            'temperature': ['mean', 'std', 'max'],
            'precipitation': ['mean', 'std', 'max'],
            'extreme_events': 'sum'
        }).reset_index()


        climate_metrics.columns = ['location', 'avg_temp', 'temp_std', 'max_temp',
                                   'avg_precip', 'precip_std', 'max_precip', 'total_extreme_events']


        climate_metrics['temp_std'] = climate_metrics['temp_std'].replace(0, 0.001)
        climate_metrics['precip_std'] = climate_metrics['precip_std'].replace(0, 0.001)


        location_risk = pd.merge(climate_metrics, self.risk_index_data, on='location')

        self.processed_data = pd.merge(self.supply_chain_data, location_risk, on='location')

        return self.processed_data

    def calculate_risk_scores(self):
        """Calculate risk scores for suppliers"""
        if self.processed_data is None:
            self.preprocess_data()


        risk_weights = {
            'temperature': 0.25,
            'precipitation': 0.20,
            'extreme_events': 0.30,
            'vulnerability': 0.15,
            'water_stress': 0.10
        }

        df = self.processed_data.copy()
        df['temp_risk'] = (df['max_temp'] - df['avg_temp']) / df['temp_std']
        df['precip_risk'] = (df['max_precip'] - df['avg_precip']) / df['precip_std']
        df['extreme_events_risk'] = df['total_extreme_events'] / df['total_extreme_events'].max()


        df = df.replace([np.inf, -np.inf], 0)


        df['overall_risk_score'] = (
            df['temp_risk'] * risk_weights['temperature'] +
            df['precip_risk'] * risk_weights['precipitation'] +
            df['extreme_events_risk'] * risk_weights['extreme_events'] +
            df['climate_vulnerability'] * risk_weights['vulnerability'] +
            df['water_stress'] * risk_weights['water_stress']
        )


        min_risk = df['overall_risk_score'].min()
        max_risk = df['overall_risk_score'].max()
        if max_risk > min_risk:
            df['overall_risk_score'] = (df['overall_risk_score'] - min_risk) / (max_risk - min_risk)
        else:
            df['overall_risk_score'] = 0.5

        df['risk_level'] = pd.cut(df['overall_risk_score'],
                                 bins=[0, 0.3, 0.7, 1],
                                 labels=['Low', 'Medium', 'High'])

        self.processed_data = df
        return self.processed_data

preprocessor = DataPreprocessor(climate_df, supply_chain_df, risk_index_df)
processed_data = preprocessor.calculate_risk_scores()

print("Data preprocessing completed!")
print("\nProcessed Data Sample:")
print(processed_data[['supplier_id', 'location', 'material_type', 'overall_risk_score', 'risk_level']].head())

print(f"\nRisk Level Distribution:")
print(processed_data['risk_level'].value_counts())


Data preprocessing completed!

Processed Data Sample:
    supplier_id location   material_type  overall_risk_score risk_level
0  Supplier_001    China       Equipment            0.236768        Low
1  Supplier_002    Japan       Logistics            0.228748        Low
2  Supplier_003  Germany          Metals            0.382839     Medium
3  Supplier_004      USA  Petrochemicals            1.000000       High
4  Supplier_005  Germany        Plastics            0.382839     Medium

Risk Level Distribution:
risk_level
Medium    11
Low        9
High       4
Name: count, dtype: int64


In [ ]:
import pandas as pd

processed_data = pd.DataFrame({
    'supplier_id': [1, 2, 3, 4, 5],
    'location': ['USA', 'India', 'India', 'China', 'USA'],
    'overall_risk_score': [0.3, 0.7, 0.5, 0.9, 0.4],
    'risk_level': ['Low', 'High', 'Medium', 'High', 'Low'],
    'material_type': ['Steel', 'Plastic', 'Plastic', 'Steel', 'Copper'],
    'revenue_impact': [100, 200, 150, 300, 120],
    'transport_mode': ['Air', 'Sea', 'Road', 'Air', 'Sea'],
    'lead_time': [5, 20, 7, 10, 25]
})


visualizer = DataVisualizer(processed_data)
print("Creating visualizations...")

visualizer.plot_risk_distribution()
visualizer.plot_risk_by_location()
visualizer.plot_material_risk()
visualizer.plot_transport_analysis()


Creating visualizations...


In [ ]:
class RiskModeler:
    """Build ML models for risk prediction"""

    def __init__(self, processed_data):
        self.processed_data = processed_data
        self.models = {}
        self.scaler = StandardScaler()

    def prepare_data_for_ml(self):
        """Prepare data for ML models"""
        df = self.processed_data.copy()

        feature_cols = ['lead_time', 'revenue_impact', 'overall_risk_score']

        X = df[feature_cols]


        y_class = (df['risk_level'] == 'High').astype(int)
        y_reg = df['overall_risk_score']

        return X, y_class, y_reg

    def train_classification_model(self):
        """Train Random Forest classifier"""
        X, y_class, _ = self.prepare_data_for_ml()
        X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.3, random_state=42)

        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)

        y_pred = rf_model.predict(X_test)
        print("Random Forest Classification Report:")
        print(classification_report(y_test, y_pred))

        self.models['classifier'] = rf_model
        self.models['feature_names'] = X.columns.tolist()
        return rf_model

    def train_regression_model(self):
        """Train Random Forest regressor"""
        X, _, y_reg = self.prepare_data_for_ml()
        X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.3, random_state=42)

        rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_regressor.fit(X_train, y_train)

        y_pred = rf_regressor.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = rf_regressor.score(X_test, y_test)

        print(f"Random Forest Regression MSE: {mse:.4f}")
        print(f"Random Forest Regression R²: {r2:.4f}")

        self.models['regressor'] = rf_regressor
        return rf_regressor

    def plot_feature_importance(self):
        """Plot feature importance"""
        if 'classifier' not in self.models:
            self.train_classification_model()

        feature_importance = pd.DataFrame({
            'feature': self.models['feature_names'],
            'importance': self.models['classifier'].feature_importances_
        }).sort_values('importance', ascending=False)

        fig = px.bar(feature_importance, x='importance', y='feature',
                     title='Feature Importance for Risk Classification', orientation='h')
        fig.show()
        return feature_importance

    def perform_clustering(self):
        """Perform K-Means clustering on suppliers"""
        cluster_features = ['lead_time', 'overall_risk_score', 'revenue_impact']
        cluster_data = self.processed_data[cluster_features].copy()

        cluster_scaled = self.scaler.fit_transform(cluster_data)

        kmeans = KMeans(n_clusters=3, random_state=42)
        self.processed_data['risk_cluster'] = kmeans.fit_predict(cluster_scaled)

        cluster_analysis = self.processed_data.groupby('risk_cluster').agg({
            'overall_risk_score': 'mean',
            'lead_time': 'mean',
            'revenue_impact': 'mean'
        }).reset_index()

        print("Cluster Analysis:")
        print(cluster_analysis)

        fig = px.scatter(self.processed_data, x='lead_time', y='overall_risk_score',
                         color='risk_cluster', size='revenue_impact',
                         hover_data=['location', 'material_type'],
                         title='Supplier Clusters by Risk Features')
        fig.show()

        return cluster_analysis


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

class ScenarioAnalyzer:
    """Analyze climate change scenarios"""

    def __init__(self, processed_data):
        self.processed_data = processed_data.copy()
        self.scenarios = {
            'Baseline': {'temp_increase': 0, 'precip_increase': 0, 'extreme_events_increase': 0},
            '1.5°C Warming': {'temp_increase': 1.5, 'precip_increase': 0.1, 'extreme_events_increase': 0.2},
            '2.0°C Warming': {'temp_increase': 2.0, 'precip_increase': 0.15, 'extreme_events_increase': 0.3},
            'Extreme Scenario': {'temp_increase': 3.0, 'precip_increase': 0.25, 'extreme_events_increase': 0.5}
        }

    def calculate_scenario_risk(self, scenario_params):
        """Calculate risk under a specific scenario"""
        scenario_data = self.processed_data.copy()

        scenario_data['avg_temp'] += scenario_params['temp_increase']
        scenario_data['max_temp'] += scenario_params['temp_increase']
        scenario_data['avg_precip'] *= (1 + scenario_params['precip_increase'])
        scenario_data['max_precip'] *= (1 + scenario_params['precip_increase'])
        scenario_data['total_extreme_events'] *= (1 + scenario_params['extreme_events_increase'])

        scenario_data['temp_risk'] = (scenario_data['max_temp'] - scenario_data['avg_temp']) / scenario_data['temp_std']
        scenario_data['precip_risk'] = (scenario_data['max_precip'] - scenario_data['avg_precip']) / scenario_data['precip_std']
        scenario_data['extreme_events_risk'] = scenario_data['total_extreme_events'] / self.processed_data['total_extreme_events'].max()

        scenario_data = scenario_data.replace([np.inf, -np.inf], 0)

        risk_weights = {
            'temperature': 0.25,
            'precipitation': 0.20,
            'extreme_events': 0.30,
            'vulnerability': 0.15,
            'water_stress': 0.10
        }

        scenario_data['overall_risk_score'] = (
            scenario_data['temp_risk'] * risk_weights['temperature'] +
            scenario_data['precip_risk'] * risk_weights['precipitation'] +
            scenario_data['extreme_events_risk'] * risk_weights['extreme_events'] +
            scenario_data['climate_vulnerability'] * risk_weights['vulnerability'] +
            scenario_data['water_stress'] * risk_weights['water_stress']
        )


        min_risk = scenario_data['overall_risk_score'].min()
        max_risk = scenario_data['overall_risk_score'].max()
        if max_risk > min_risk:
            scenario_data['overall_risk_score'] = (scenario_data['overall_risk_score'] - min_risk) / (max_risk - min_risk)

        return scenario_data['overall_risk_score']

    def analyze_all_scenarios(self):
        """Analyze all climate scenarios"""
        scenario_results = {}

        for scenario_name, scenario_params in self.scenarios.items():
            risk_scores = self.calculate_scenario_risk(scenario_params)
            scenario_results[scenario_name] = risk_scores.mean()

        scenario_df = pd.DataFrame.from_dict(scenario_results, orient='index', columns=['Average Risk Score']).reset_index()
        scenario_df = scenario_df.rename(columns={'index': 'Scenario'})

        fig = px.bar(scenario_df, x='Scenario', y='Average Risk Score',
                     title='Average Supply Chain Risk Under Different Climate Scenarios',
                     color='Average Risk Score', color_continuous_scale='Viridis')
        fig.show()

        return scenario_df

    def supplier_scenario_analysis(self, supplier_id):
        """Analyze scenarios for a specific supplier"""
        supplier_risks = {}

        for scenario_name, scenario_params in self.scenarios.items():
            risk_scores = self.calculate_scenario_risk(scenario_params)
            supplier_data = self.processed_data[self.processed_data['supplier_id'] == supplier_id]
            if not supplier_data.empty:
                supplier_idx = supplier_data.index[0]
                supplier_risks[scenario_name] = risk_scores.iloc[supplier_idx - risk_scores.index[0]]

        if supplier_risks:
            scenario_df = pd.DataFrame.from_dict(supplier_risks, orient='index', columns=['Risk Score']).reset_index()
            scenario_df = scenario_df.rename(columns={'index': 'Scenario'})

            fig = px.line(scenario_df, x='Scenario', y='Risk Score',
                          title=f'Risk Score for {supplier_id} Under Different Scenarios',
                          markers=True)
            fig.show()

            return scenario_df
        else:
            print(f"Supplier {supplier_id} not found.")
            return None


np.random.seed(42)
processed_data = pd.DataFrame({
    'supplier_id': [f"S{i}" for i in range(1, 6)],
    'avg_temp': np.random.uniform(20, 30, 5),
    'max_temp': np.random.uniform(30, 40, 5),
    'temp_std': np.random.uniform(1, 5, 5),
    'avg_precip': np.random.uniform(50, 200, 5),
    'max_precip': np.random.uniform(200, 300, 5),
    'precip_std': np.random.uniform(5, 15, 5),
    'total_extreme_events': np.random.randint(1, 10, 5),
    'climate_vulnerability': np.random.uniform(0, 1, 5),
    'water_stress': np.random.uniform(0, 1, 5)
})

print("Dummy processed_data:")
print(processed_data)


print("\nRunning scenario analysis...")
scenario_analyzer = ScenarioAnalyzer(processed_data)
scenario_results = scenario_analyzer.analyze_all_scenarios()

print("\nScenario Analysis Results:")
print(scenario_results)


sample_supplier = processed_data['supplier_id'].iloc[0]
print(f"\nAnalyzing scenarios for {sample_supplier}...")
supplier_scenario_results = scenario_analyzer.supplier_scenario_analysis(sample_supplier)




Dummy processed_data:
  supplier_id   avg_temp   max_temp  temp_std  avg_precip  max_precip  \
0          S1  23.745401  31.559945  1.082338   77.510676  261.185289   
1          S2  29.507143  30.580836  4.879639   95.636336  213.949386   
2          S3  27.319939  38.661761  4.329771  128.713465  229.214465   
3          S4  25.986585  36.011150  1.849356  114.791753  236.636184   
4          S5  21.560186  37.080726  1.727300   93.684371  245.606998   

   precip_std  total_extreme_events  climate_vulnerability  water_stress  
0   12.851760                     3               0.013265      0.230894  
1    6.996738                     7               0.942202      0.241025  
2   10.142344                     5               0.563288      0.683264  
3   10.924146                     9               0.385417      0.609997  
4    5.464504                     7               0.015966      0.833195  

Running scenario analysis...



Scenario Analysis Results:
           Scenario  Average Risk Score
0          Baseline            0.345480
1     1.5°C Warming            0.345813
2     2.0°C Warming            0.345963
3  Extreme Scenario            0.346237

Analyzing scenarios for S1...


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class SupplyChainVisualizer:
    def __init__(self, processed_data):
        self.processed_data = processed_data
        self.graph = nx.Graph()
        self.build_network()

    def build_network(self):

        possible_loc_cols = ['location', 'region', 'city', 'country', 'site']
        loc_col = None
        for col in possible_loc_cols:
            if col in self.processed_data.columns:
                loc_col = col
                break

        for _, row in self.processed_data.iterrows():
            supplier = row['supplier_id']
            risk = row['overall_risk_score']
            material = row['material_type']


            self.graph.add_node(supplier,
                                risk=risk,
                                type='supplier',
                                material=material)

            if loc_col:
                location = row[loc_col]
                self.graph.add_node(location, type='location')
                self.graph.add_edge(supplier, location)

            self.graph.add_node(material, type='material')
            self.graph.add_edge(supplier, material)

    def visualize_network(self):
        pos = nx.spring_layout(self.graph, k=0.5)
        plt.figure(figsize=(12, 8))


        node_types = nx.get_node_attributes(self.graph, 'type')
        risks = nx.get_node_attributes(self.graph, 'risk')


        node_colors = []
        for n in self.graph.nodes():
            if node_types.get(n) == 'supplier':
                node_colors.append(risks.get(n, 0))
            elif node_types.get(n) == 'material':
                node_colors.append(0.5)
            else:
                node_colors.append(0.2)

        nx.draw(self.graph, pos,
                with_labels=True,
                node_size=800,
                node_color=node_colors,
                cmap=plt.cm.Reds,
                font_size=8,
                font_color="black")

        plt.title("Supply Chain Network Visualization")
        plt.show()



In [ ]:
import pandas as pd

class RiskAdvisor:
    """Generate risk mitigation recommendations"""

    def __init__(self, processed_data):
        self.processed_data = processed_data.copy()

        if 'risk_level' not in self.processed_data.columns:
            self.processed_data['risk_level'] = self.processed_data['overall_risk_score'].apply(
                lambda x: 'High' if x > 0.7 else ('Medium' if x > 0.4 else 'Low')
            )

    def generate_supplier_recommendations(self, supplier_id):
        """Generate recommendations for a specific supplier"""
        supplier_data = self.processed_data[self.processed_data['supplier_id'] == supplier_id]

        if supplier_data.empty:
            return [f"Supplier {supplier_id} not found."]

        row = supplier_data.iloc[0]
        recommendations = []

        risk_score = row['overall_risk_score']
        if risk_score > 0.7:
            recommendations.append("🚨 CRITICAL RISK: Immediate action required")
        elif risk_score > 0.4:
            recommendations.append("⚠️ MODERATE RISK: Develop mitigation plan")
        else:
            recommendations.append("✅ LOW RISK: Maintain current monitoring")

        if 'climate_vulnerability' in row and row['climate_vulnerability'] > 0.7:
            recommendations.append(f"📍 High climate vulnerability in {row.get('location', 'Unknown')}: Consider diversifying sourcing")
        if 'water_stress' in row and row['water_stress'] > 0.6:
            recommendations.append(f"💧 High water stress in {row.get('location', 'Unknown')}: Assess water dependency")


        if 'infrastructure_quality' in row and row['infrastructure_quality'] < 0.5:
            recommendations.append("🏗️ Poor infrastructure quality: Develop contingency plans")
        if 'recovery_capacity' in row and row['recovery_capacity'] < 0.5:
            recommendations.append("🔄 Low recovery capacity: Enhance business continuity planning")

        transport = row.get('transport_mode', None)
        if transport == 'Sea':
            recommendations.append("🚢 Sea transport: Monitor port disruptions and weather patterns")
        elif transport == 'Air':
            recommendations.append("✈️ Air transport: Consider carbon footprint and weather delays")

        if row.get('single_source', False):
            recommendations.append("🔗 Single source dependency: Identify alternative suppliers")

        if row.get('lead_time', 0) > 60:
            recommendations.append("⏰ Long lead time: Build buffer inventory for critical materials")


        material = row.get('material_type', 'Unknown')
        if material == 'Petrochemicals':
            recommendations.append("🛢️ Petrochemicals: Monitor temperature-sensitive storage and transport")
        elif material == 'Metals':
            recommendations.append("⚙️ Metals: Consider corrosion protection in humid climates")
        elif material == 'Equipment':
            recommendations.append("🔧 Equipment: Ensure maintenance protocols for extreme weather")

        return recommendations

    def generate_summary_report(self):
        """Generate comprehensive risk assessment report"""
        high_risk = self.processed_data[self.processed_data['risk_level'] == 'High']
        medium_risk = self.processed_data[self.processed_data['risk_level'] == 'Medium']
        low_risk = self.processed_data[self.processed_data['risk_level'] == 'Low']


        total_suppliers = len(self.processed_data)
        high_risk_pct = len(high_risk) / total_suppliers * 100 if total_suppliers else 0
        medium_risk_pct = len(medium_risk) / total_suppliers * 100 if total_suppliers else 0
        low_risk_pct = len(low_risk) / total_suppliers * 100 if total_suppliers else 0

        risk_by_location = (
            self.processed_data.groupby('location')['overall_risk_score'].mean().sort_values(ascending=False)
            if 'location' in self.processed_data.columns else pd.Series()
        )
        risk_by_material = (
            self.processed_data.groupby('material_type')['overall_risk_score'].mean().sort_values(ascending=False)
            if 'material_type' in self.processed_data.columns else pd.Series()
        )

        high_risk_revenue_impact = high_risk['revenue_impact'].sum() if 'revenue_impact' in self.processed_data.columns else 0
        total_revenue_impact = self.processed_data['revenue_impact'].sum() if 'revenue_impact' in self.processed_data.columns else 0
        revenue_at_risk_pct = (high_risk_revenue_impact / total_revenue_impact * 100) if total_revenue_impact else 0

        report = f"""
# SUPPLY CHAIN CLIMATE RISK ASSESSMENT REPORT
## Executive Summary

### Risk Distribution
- Total Suppliers Analyzed: {total_suppliers}
- High-Risk Suppliers: {len(high_risk)} ({high_risk_pct:.1f}%)
- Medium-Risk Suppliers: {len(medium_risk)} ({medium_risk_pct:.1f}%)
- Low-Risk Suppliers: {len(low_risk)} ({low_risk_pct:.1f}%)

### Financial Exposure
- Revenue at Risk (High-Risk Suppliers): ${high_risk_revenue_impact:.1f}M ({revenue_at_risk_pct:.1f}%)
- Total Revenue Exposure: ${total_revenue_impact:.1f}M

### Highest Risk Locations
{risk_by_location.head(3).to_string() if not risk_by_location.empty else "No location data available"}

### Highest Risk Material Types
{risk_by_material.head(3).to_string() if not risk_by_material.empty else "No material data available"}

### Key Findings
1. Climate vulnerability is the primary risk driver for most suppliers
2. Single-source suppliers represent {len(self.processed_data[self.processed_data.get('single_source', False)])} out of {total_suppliers} total suppliers
3. Sea transport mode carries additional weather-related risks
4. Locations with high water stress require immediate attention

### Strategic Recommendations
1. **Immediate Actions (0-3 months)**
   - Conduct detailed assessment of {len(high_risk)} high-risk suppliers
   - Develop contingency plans for single-source dependencies
   - Establish climate monitoring systems for critical locations

2. **Short-term Actions (3-12 months)**
   - Diversify supplier base in high-risk locations
   - Implement climate-resilient logistics strategies
   - Build strategic inventory buffers for critical materials

3. **Long-term Actions (1-3 years)**
   - Integrate climate risk into supplier selection criteria
   - Develop partnerships with suppliers on climate adaptation
   - Invest in climate-resilient infrastructure and technologies

### Monitoring and Reporting
- Quarterly risk assessment updates
- Monthly monitoring of high-risk supplier performance
- Annual climate scenario stress testing
        """

        return report


In [ ]:
import pandas as pd

class ResultsExporter:
    """Export analysis results and visualizations"""

    def __init__(self, processed_data, scenario_results):
        self.processed_data = processed_data.copy()
        self.scenario_results = scenario_results.copy()


        if 'risk_level' not in self.processed_data.columns:
            self.processed_data['risk_level'] = self.processed_data['overall_risk_score'].apply(
                lambda x: 'High' if x > 0.7 else ('Medium' if x > 0.4 else 'Low')
            )

    def export_data(self):
        """Export data to CSV files"""

        self.processed_data.to_csv('supply_chain_risk_assessment.csv', index=False)
        print("Exported: supply_chain_risk_assessment.csv")


        self.scenario_results.to_csv('climate_scenario_analysis.csv', index=False)
        print("Exported: climate_scenario_analysis.csv")

        high_risk_summary = self.processed_data[self.processed_data['risk_level'] == 'High'][
            ['supplier_id', 'location', 'material_type', 'transport_mode',
             'overall_risk_score', 'revenue_impact', 'single_source']
        ]
        high_risk_summary.to_csv('high_risk_suppliers.csv', index=False)
        print("Exported: high_risk_suppliers.csv")


        if 'location' in self.processed_data.columns:
            location_summary = self.processed_data.groupby('location').agg({
                'overall_risk_score': 'mean',
                'supplier_id': 'count',
                'revenue_impact': 'sum'
            }).reset_index().rename(columns={'supplier_id': 'supplier_count'})
            location_summary.to_csv('risk_by_location.csv', index=False)
            print("Exported: risk_by_location.csv")
        else:
            print("⚠️ No 'location' column found, skipping risk_by_location.csv")
            location_summary = pd.DataFrame()

        return ["supply_chain_risk_assessment.csv",
                "climate_scenario_analysis.csv",
                "high_risk_suppliers.csv"] + (["risk_by_location.csv"] if not location_summary.empty else [])

    def generate_dashboard_summary(self):
        """Generate key metrics for dashboard"""
        total_suppliers = len(self.processed_data)
        high_risk_count = len(self.processed_data[self.processed_data['risk_level'] == 'High'])
        avg_risk_score = self.processed_data['overall_risk_score'].mean()
        total_revenue_impact = self.processed_data['revenue_impact'].sum()

        dashboard_metrics = {
            'total_suppliers': total_suppliers,
            'high_risk_suppliers': high_risk_count,
            'high_risk_percentage': round(high_risk_count/total_suppliers*100, 1) if total_suppliers else 0,
            'average_risk_score': round(avg_risk_score, 3),
            'total_revenue_exposure': round(total_revenue_impact, 1),
            'locations_analyzed': len(self.processed_data['location'].unique()) if 'location' in self.processed_data.columns else 0,
            'material_types': len(self.processed_data['material_type'].unique()) if 'material_type' in self.processed_data.columns else 0
        }

        return dashboard_metrics



In [ ]:
class ResultsExporter:
    """Export analysis results and visualizations"""

    def __init__(self, processed_data, scenario_results):
        self.processed_data = processed_data
        self.scenario_results = scenario_results

    def export_data(self):
        """Export data to CSV files"""
        exported_files = []

        try:
            self.processed_data.to_csv('supply_chain_risk_assessment.csv', index=False)
            print("Exported: supply_chain_risk_assessment.csv")
            exported_files.append("supply_chain_risk_assessment.csv")
        except Exception as e:
            print(f"⚠️ Failed to export risk assessment: {e}")

        try:
            self.scenario_results.to_csv('climate_scenario_analysis.csv', index=False)
            print("Exported: climate_scenario_analysis.csv")
            exported_files.append("climate_scenario_analysis.csv")
        except Exception as e:
            print(f"⚠️ Failed to export scenario results: {e}")


        if "risk_level" in self.processed_data.columns:
            try:
                high_risk_summary = self.processed_data[self.processed_data['risk_level'] == 'High'][
                    [col for col in ['supplier_id', 'country', 'location', 'material_type',
                                     'transport_mode', 'overall_risk_score', 'revenue_impact', 'single_source']
                     if col in self.processed_data.columns]
                ]
                high_risk_summary.to_csv('high_risk_suppliers.csv', index=False)
                print("Exported: high_risk_suppliers.csv")
                exported_files.append("high_risk_suppliers.csv")
            except Exception as e:
                print(f"⚠️ Failed to export high risk summary: {e}")


        loc_col = None
        for candidate in ['location', 'country', 'region']:
            if candidate in self.processed_data.columns:
                loc_col = candidate
                break

        if loc_col:
            try:
                location_summary = self.processed_data.groupby(loc_col).agg({
                    'overall_risk_score': 'mean' if 'overall_risk_score' in self.processed_data else 'count',
                    'supplier_id': 'count' if 'supplier_id' in self.processed_data else 'size',
                    'revenue_impact': 'sum' if 'revenue_impact' in self.processed_data else 'size'
                }).reset_index().rename(columns={'supplier_id': 'supplier_count'})
                location_summary.to_csv('risk_by_location.csv', index=False)
                print("Exported: risk_by_location.csv")
                exported_files.append("risk_by_location.csv")
            except Exception as e:
                print(f"⚠️ Failed to export risk by location: {e}")

        return exported_files

    def generate_dashboard_summary(self):
        """Generate key metrics for dashboard"""
        total_suppliers = len(self.processed_data)


        if "risk_level" in self.processed_data.columns:
            high_risk_count = len(self.processed_data[self.processed_data['risk_level'] == 'High'])
        else:
            high_risk_count = 0

        avg_risk_score = self.processed_data['overall_risk_score'].mean() if 'overall_risk_score' in self.processed_data else 0
        total_revenue_impact = self.processed_data['revenue_impact'].sum() if 'revenue_impact' in self.processed_data else 0


        loc_col = None
        for candidate in ['location', 'country', 'region']:
            if candidate in self.processed_data.columns:
                loc_col = candidate
                break

        locations_analyzed = self.processed_data[loc_col].nunique() if loc_col else 1
        material_types = self.processed_data['material_type'].nunique() if 'material_type' in self.processed_data else 1

        dashboard_metrics = {
            'total_suppliers': total_suppliers,
            'high_risk_suppliers': high_risk_count,
            'high_risk_percentage': round(high_risk_count/total_suppliers*100, 1) if total_suppliers > 0 else 0,
            'average_risk_score': round(avg_risk_score, 3),
            'total_revenue_exposure': round(total_revenue_impact, 1),
            'locations_analyzed': locations_analyzed,
            'material_types': material_types
        }

        return dashboard_metrics

